**Importazione delle librerie utili allo sviluppo dei modelli**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os as os
from itertools import product
from itertools import islice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import math
import scipy.stats as sts
from itertools import chain

# Anonymize DF
from anonymizedf.anonymizedf import anonymize
import dataframe_image as dfi

from pandas import concat
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from sklearn.metrics import mean_squared_log_error
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd


import warnings
warnings.filterwarnings('ignore')

#import custom function to explore dataset values and plot
#from src.explore_functions import df_value_stat

#Inizialize constants
RANDOM_SEED = 42
SAMPLE_NUMBER = 5
N_COMB = 10

#Dataset source commentare uno o l'altro dipende dalla posizione del dataset
#DATASET_SOURCE = "dataset.xlsx"
#SHEET_NAME = "Sheet1"

**Settaggio di alcune opzioni grafiche**

In [ ]:
sns.set()
sns.set_style("darkgrid")
sns.set_palette("inferno_r")
%matplotlib inline
%config InlineBackend.figure_format = "retina"
plt.rcParams['figure.figsize'] = (15.0, 10.0)

**Rinomino i clienti in modo più comprensibile**

In [ ]:
def rinomina_clienti(df_input):
    anon = anonymize(df_input)
    anon.fake_categories("Cliente")
    remap = {}
    fake_cliente = df_input['Fake_Cliente'].unique().tolist()
    cliente = df_input['Cliente'].unique().tolist()
    for key in fake_cliente:
        for value in cliente:
            remap[key] = value
            cliente.remove(value)
            break
    display(remap)
    df_input = df_input.assign(Cliente = df['Fake_Cliente'])
    df_input = df_input.drop(columns='Fake_Cliente')
    return df_input, remap

**Funzione che permette di tornare ad ottenere i clienti col proprio ID/nome**

In [ ]:
def rinomina_clienti_back(df_input, remap):
    df_input['Cliente'] = df_input['Cliente'].map(remap)
    return df_input

In [ ]:
df, remap = rinomina_clienti(df_input=df)
df

# Modelling

**Analisi delle componenti del dataset per variabile d'interesse**

In [ ]:
def makeDataLagged(data, n_in=1, n_out=1, dropnan=True):
    """
    Lagga tutte le colonne nei dati
    Argomenti:
    data: sequenza di osservazioni, lista o Numpy array.
    n_in: Numero di osservazioni laggate come input (X).
    n_out: Numero di osservazioni come output (y).
    dropnan: Booleana, sceglie se eliminare o no le righe con valori NaN.
    Ritorna:
    Pandas dataframe o np array
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
        # sequenza input (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        # sequenza di forecast (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        # mette tutto insieme
        agg = concat(cols, axis=1)
        agg.columns = names
        # droppa le righe coi valori NaN
    if dropnan:
        agg.dropna(inplace=True)
    
    return agg

### Automatizzazione del processo di allenamento dei modelli e sviluppo dei forecast con funzioni e loop

Scelgo il range di tempo su cui fare previsioni, i modelli e gli iperparametri su cui fare il tuning

In [ ]:
# Creazione del nuovo indice con intervallo mensile compreso tra '2020-01-01' e '2022-12-01'
#AGGIORNARE NEL CASO SI VOGLIA ANDARE AVANTI CON LE PREVISIONI
new_index = pd.date_range(start='2020-01-01', end='2022-12-01', freq='MS')

# define the models
models = {'Random Forest': RandomForestRegressor(),
          'XG Boost': xgb.XGBRegressor(verbosity=0 ,random_state=RANDOM_SEED),
          'Ada Boost': AdaBoostRegressor(random_state=RANDOM_SEED),
          'K-nn': KNeighborsRegressor(n_jobs=-1),
          'Decision Tree': DecisionTreeRegressor(random_state=RANDOM_SEED)}

# Hyperparameter tuning using RandomizedSearchCV
param_grid = {'Random Forest': {'n_estimators': [10, 20, 50, 100],
                                'max_features': ['sqrt', 'log2'],
                                'max_depth' : [i for i in range(5,25)]
                                },
              'XG Boost': {'n_estimators': [10, 20, 50, 100],
                           'max_features': ['sqrt', 'log2'],
                           'max_depth' : [i for i in range(5,25)]
                          },
              'Ada Boost' : {'n_estimators': [10, 20, 50, 100],
                             'loss': ['linear', 'square','exponential'],
                             'learning_rate' : [i for i in [0.1,0.3,0.5,0.7,0.9,1]]
                            },
              'K-nn': {'n_neighbors': [1,2,3],
                       'weights': ['uniform', 'distance']
                      },
              'Decision Tree': {'criterion': ['squared_error', 'absolute_error','friedman_mse'],
                                'max_features': ["auto", "sqrt", "log2"],
                                'max_depth' : [i for i in range(5,25)]
                                }
             }

**Creazione delle funzioni per ottenere i dataframe con le varie combinazioni di feature, e creare una lista di dataframe con questi**

In [ ]:
def makeBestComb(df, features, N_COMB = 10):
    '''
    Calcolo il numero di vendite e occorenze per ogni combinazione possibile dato un insieme di features
     
    df : dataframe su cui trova combinazioni e calcola la somma e l occorenza per ogni combinazione
    features : insieme di feature, aggrega per ogni combinazione di queste partendo da 1 a n_feature
    N_COMB : numero di combinazioni migliori da prendere per ogni combinazione di features
   
    Restituisce una lista di dataframe
    '''
    i = 0
    var = list()
    combo = list()
    somma_tot = df['Vendite_consuntive'].sum()
    count_tot = df['Vendite_consuntive'].count()
    
    for feature in features:
        var.append(feature)
        combo.append(df.groupby(var)['Vendite_consuntive'].agg(['sum', 'count']).sort_values(by='sum', ascending=False).reset_index())
        
        display(combo[i].head(N_COMB))
        print("Somma Vendite Consuntive Considerate")
        print(combo[i]['sum'].head(N_COMB).sum())
        print("Somma Occorrenze Considerate")
        print(combo[i]['count'].head(N_COMB).sum())
        print("Rapporto Vendite Consuntive Totali/Considerate")
        print(combo[i]['sum'].head(N_COMB).sum()/somma_tot)
        print("Rapporto Occorrenze Totali/Considerate")
        print(combo[i]['count'].head(N_COMB).sum()/count_tot)
        
        i += 1
    return combo

In [ ]:
def makeDataframeFromCombo(df, combo, features, N_COMB = 10):
    '''
    Per ogni top combinazione trovata prima, crea un dataframe contenente tutte le transazioni con quella combinazione
    
    df : Dataframe da cui prende le transazioni
    combo : dataframe contente le combinazioni top (+ vendite tot/+ occorrenze)
    features : insieme di feature, aggrega per ogni combinazione di queste partendo da 1 a n_feature
    N_COMB : numero di combinazioni migliori da prendere per ogni combinazione di features
    
    Restituisce una matrice di dataframe con tutte le combinazioni
    Il primo indice indica quante feature sono prese in considerazioni per la combinazione
    Il secondo indice indica quale tra le top N_COMB prendere
    --> df_comb[4][0] --> Prende il dataframe della combinazione con più vendite [0], 
    tra quelli con la combinazioni di 5 features [4]
    '''
    df_comb = list()
    df_comb.append(list())
    x = 0
    for i in range(N_COMB):
        df_comb[x].append(df[df[features[x]] == combo[x].iloc[i][x]])
    x += 1
    df_comb.append(list())
    for i in range(N_COMB):
        df_comb[x].append(df[(df[features[x-1]] == combo[x].iloc[i][x-1]) &
                     (df[features[x]] == combo[x].iloc[i][x])])
    x += 1
    df_comb.append(list())
    for i in range(N_COMB):
        df_comb[x].append(df[(df[features[x-2]] == combo[x].iloc[i][x-2]) &
                     (df[features[x-1]] == combo[x].iloc[i][x-1]) &
                     (df[features[x]] == combo[x].iloc[i][x])])
    x += 1
    df_comb.append(list())
    for i in range(N_COMB):
        df_comb[x].append(df[(df[features[x-3]] == combo[x].iloc[i][x-3]) &
                     (df[features[x-2]] == combo[x].iloc[i][x-2]) &
                     (df[features[x-1]] == combo[x].iloc[i][x-1]) &
                     (df[features[x]] == combo[x].iloc[i][x])])
    x += 1
    df_comb.append(list())
    for i in range(N_COMB):
        df_comb[x].append(df[(df[features[x-4]] == combo[x].iloc[i][x-4]) &
                     (df[features[x-3]] == combo[x].iloc[i][x-3]) &
                     (df[features[x-2]] == combo[x].iloc[i][x-2]) &
                     (df[features[x-1]] == combo[x].iloc[i][x-1]) &
                     (df[features[x]] == combo[x].iloc[i][x])])
        
    return df_comb

**Reindicizzazione delle date, così da ottenere date ordinate senza buchi, creando una lista di dataframe col groupby delle variabili d'interesse**

In [ ]:
def makeGroupByCombo(dataframe_combinazioni, N_COMB = 10, N_FEAT_COMB = 4):
    '''
    Scelgo quante feature considerare per le combinazioni e quante combinazioni prendere 
    Per ogni combinazione aggruppa le vendite per mese e riempe con 0 in mesi senza vendite
    '''
    d = list()
    for i in range(0,N_COMB):#il valore centrale dipende da quante combinazioni teniamo in considerazione: nel nostro caso 10(0->9)
        d.append(dataframe_combinazioni[N_FEAT_COMB][i])

        d[i] = d[i].groupby([d[i].index,'Divisione', 'Sub_pool', 'Cliente', 
                                'Mezzo_trasporto', 'Funzione_commerciale'])[['Vendite_consuntive','Vendite_programmate']].sum()
        d[i] = d[i].reset_index()
        d[i] = d[i].set_index('Data')
        d[i] = d[i][['Divisione', 'Sub_pool', 'Cliente', 'Mezzo_trasporto', 
                     'Funzione_commerciale','Vendite_consuntive','Vendite_programmate']]
        
        #Se non è presente almeno una vendita per ogni mese, aggiungo vendite a 0 per i mesi mancanti
        if d[0].shape[0] > d[i].shape[0]:     
            # Reindicizzazione del DataFrame con i valori mancanti sostituiti da NaN
            d[i] = d[i].reindex(new_index)
            d[i] = d[i].reset_index()
            d[i] = d[i].rename(columns={"index": "Data"})
            d[i] = d[i].set_index('Data')
            # Sostituzione di NaN con 0
            d[i] = d[i].fillna(0)
    return d

**Le date nuove sono state riempite con degli zeri partendo dall'assunzione che non ci siano state vendite nei mesi mancanti**

In [ ]:
features = ['Divisione', 'Sub_pool', 'Cliente', 'Mezzo_trasporto', 'Funzione_commerciale']
N_COMB = 10
N_FEAT_COMB = 4
combo = makeBestComb(df, features, N_COMB)
df_comb = makeDataframeFromCombo(df, combo, features, N_COMB)
display(df_comb[N_FEAT_COMB][0])
df_comb_groupby = makeGroupByCombo(df_comb, N_COMB, N_FEAT_COMB)
display(df_comb_groupby[0])

**Plot  delle autocorrelazioni per capire quanti e quali lag sono significativi**

In [ ]:
for i in range(0,N_COMB,1):
    
    plot_acf(df_comb_groupby[i]['Vendite_consuntive'],title=f'Autocorrelazione dataframe {i}')

**Si denoti come in tutti i dataframe il primo lag è il più significativo, ogni tanto il secondo e di rado il terzo, da qui la scelta di usare 3 lag**

**Creazione dei dataframe laggati**

In [ ]:
def makeLagged(dataframe,n_mesi):
    '''
    Richiama funzione per lag, rimposta la data come index
    n_mesi : numero mesi di lag
    '''
    dataframe_lagged = makeDataLagged(dataframe[['Vendite_consuntive']],n_mesi)
    dataframe_lagged = dataframe_lagged.reset_index()
    dataframe_lagged = dataframe_lagged.set_index('Data')
    return dataframe_lagged

**Creazione della suddivisione dei mesi di train e test**

In [ ]:
def makeMonths(dataframe, n_mesi = 3):
    '''
    Costruisce un dizionario con coppia (mese di train : mese di test)
    Per tutti i mesi presenti nel dataframe
    n_mesi : numero mesi di train
    '''
    mesi_train = list()
    mesi_train = dataframe.index.strftime("%Y-%m").tolist()
    mesi_test = mesi_train[:]
    for i in range(n_mesi):
        mesi_test.pop(0)
    mesi = dict(zip(mesi_train, mesi_test))
    return mesi

**Estrazione di altre feature dalla serie temporale, usando il rolling**

In [ ]:
def makeRolling(dataframe_lagged, mesi_rolling = 1):
    '''
    Calcola il rolling dei mesi precedenti(media, mediana, minimo e massimo) e li salva nel dataframe
    
    '''
    dataframe_lagged['Rolled_mean']=dataframe_lagged['var1(t)'].rolling(mesi_rolling).mean()
    dataframe_lagged['Rolled_median']=dataframe_lagged['var1(t)'].rolling(mesi_rolling).median()
    dataframe_lagged['Min']=dataframe_lagged['var1(t)'].rolling(mesi_rolling).min()
    dataframe_lagged['Max']=dataframe_lagged['var1(t)'].rolling(mesi_rolling).max()
    
    return dataframe_lagged

**Funzione per allenare i modelli precedentemente scelti inserendo gli input richiesti**

In [ ]:
def makeModels(str_model, dataframe, dataframe_lagged, mesi, n_mesi_train = 3, n_mesi_lagged = 3, lag = ''):
    dataframe_lagged = makeRolling(dataframe_lagged, mesi_rolling = 1)
    
    predizioni = list()
    mae = list()
    
    #train e test per ogni modello inserito prima a dizionario
    for mese_train, mese_test in mesi.items():
        fine_mese_train = np.datetime64(mese_train) + np.timedelta64(n_mesi_train-1, 'M')
        X_train = dataframe_lagged.loc[mese_train:fine_mese_train].drop('var1(t)', axis=1).fillna(0).values
        X_test = dataframe_lagged.loc[mese_test].drop('var1(t)', axis=1).fillna(0).values
        y_train = dataframe_lagged.loc[mese_train:fine_mese_train]['var1(t)'].fillna(0).values
        y_test = dataframe_lagged.loc[mese_test]['var1(t)'].fillna(0).values
        
        #Randomized search per ottimizzazione degli iperparametri dei vari modelli
        model = models[str_model]
        param_search = param_grid[str_model]
        neg_mae = make_scorer(mean_absolute_error, greater_is_better=False)
        tscv = TimeSeriesSplit(n_splits=2)
        rs = RandomizedSearchCV(estimator=model, cv=tscv, param_distributions=param_search, scoring = neg_mae)
        rs.fit(X_train, y_train)
        best_score = rs.best_score_
        best_model = rs.best_estimator_
        y_pred = best_model.predict(X_test)
        predizioni.append(y_pred[0])
        mae.append(mean_absolute_error(y_true=y_test, y_pred=y_pred))
    
    #predizione vendite e calcolo delta
    for i in range(n_mesi_train):
        predizioni.insert(0,0)
        mae.insert(0,0)
        
    for i in range(n_mesi_lagged):
        predizioni.insert(0,0)
        mae.insert(0,0)
    if lag == '':
        dataframe['Vendite_prev_N_Mesi_'+str(n_mesi_train)] = predizioni
        dataframe['Mae_N_Mesi_'+str(n_mesi_train)] = mae
        dataframe['Delta_ass_N_Mesi_'+str(n_mesi_train)] = dataframe['Vendite_consuntive'] - dataframe['Vendite_prev_N_Mesi_'+str(n_mesi_train)]
        dataframe['Delta_perc_N_Mesi_'+str(n_mesi_train)] = ((dataframe['Vendite_consuntive'] -
        dataframe['Vendite_prev_N_Mesi_'+str(n_mesi_train)]) / dataframe['Vendite_prev_N_Mesi_'+str(n_mesi_train)]).round(3)
    else:
        dataframe['Vendite_prev_N_Mesi_'+lag+str(n_mesi_lagged)] = predizioni
        dataframe['Mae_N_Mesi_'+lag+str(n_mesi_lagged)] = mae
        dataframe['Delta_ass_N_Mesi_'+lag+str(n_mesi_lagged)] = dataframe['Vendite_consuntive'] - dataframe['Vendite_prev_N_Mesi_'+lag+str(n_mesi_lagged)]
        dataframe['Delta_perc_N_Mesi_'+lag+str(n_mesi_lagged)] = ((dataframe['Vendite_consuntive'] -
        dataframe['Vendite_prev_N_Mesi_'+lag+str(n_mesi_lagged)]) / dataframe['Vendite_prev_N_Mesi_'+lag+str(n_mesi_lagged)]).round(3) 

    return dataframe

**Calcolo della media dei forecast dei vari modelli**

In [ ]:
def makeMeanForecast(dataframe, n_mesi = 3, lag = ''):
    '''
    Funzione di calcolo della media dei forecast dei vari modelli a seconda del numero di mesi scelto, minimo 3
    '''
    
    previste = dataframe.filter(regex='prev_N_Mesi_'+lag+str(n_mesi))
    
    previste['Final_forecast'] = previste.mean(axis=1)
    
    dataframe['Final_forecast_N_Mesi_'+lag+str(n_mesi)] = previste['Final_forecast']
    
    return dataframe

**Creazione dei dataframe con i forecast per mese e forecast finale per ogni numero di mese di train**

In [ ]:
def makeForecast(dataframe, n_mesi_max = 3, n_mesi_def = 3, lag = ''):
    '''
    Funzione che crea un dataframe di vari forecast, basati sul range di mesi scelto come train (e lag) 
    '''
    df_pred = {}
    dataframe_model = dataframe.copy()
    for str_model, model in models.items():
        for n_mesi in range(3,n_mesi_max+1):
            if lag == '':
                dataframe_lagged = makeLagged(dataframe,n_mesi_def)
                mesi = makeMonths(dataframe_lagged, n_mesi)
                df_pred[str_model] = makeModels(str_model, dataframe_model, dataframe_lagged, mesi, n_mesi_train = n_mesi)

            else:
                dataframe_lagged = makeLagged(dataframe,n_mesi)
                mesi = makeMonths(dataframe_lagged, n_mesi_def)
                df_pred[str_model] = makeModels(str_model, dataframe_model, dataframe_lagged, mesi, n_mesi_lagged = n_mesi, lag = lag)
        #print(str_model)
        #display(df_pred[str_model])
                
    data = df_pred[str_model].reset_index()#così ho un indice temporale della lunghezza giusta            
                  
    df_forecast=pd.DataFrame()
    df_forecast['Data'] = data['Data'].values
    
    for str_model, model in models.items():
        for mese in range(3, n_mesi_max+1):
            df_forecast[str_model+'_Vendite_prev_N_Mesi_'+lag+str(mese)] = df_pred[str_model]['Vendite_prev_N_Mesi_'+lag+str(mese)].values
            df_forecast[str_model+'_Delta_ass_N_Mesi_'+lag+str(mese)] = df_pred[str_model]['Delta_ass_N_Mesi_'+lag+str(mese)].values
            df_forecast[str_model+'_Delta_perc_N_Mesi_'+lag+str(mese)] = df_pred[str_model]['Delta_perc_N_Mesi_'+lag+str(mese)].values
            df_forecast[str_model+'Mae_N_Mesi_'+lag+str(mese)] = df_pred[str_model]['Mae_N_Mesi_'+lag+str(mese)].values

    #Ora faccio la media dei forecast e la metto nella colonna final forecast
    df_forecast = df_forecast.set_index('Data')
    for n_mesi in range(3,n_mesi_max+1):
        df_forecast = makeMeanForecast(df_forecast, n_mesi, lag)

    dataframe = pd.concat([dataframe,df_forecast], axis=1)
    
    for n_mesi in range(3,n_mesi_max+1):
        dataframe['Delta_ass_finale_N_Mesi_'+lag+str(n_mesi)]=dataframe['Vendite_consuntive']-dataframe['Final_forecast_N_Mesi_'+lag+str(n_mesi)]
        dataframe['Delta_perc_nuovo_N_Mesi_'+lag+str(n_mesi)]=((dataframe['Vendite_consuntive']-dataframe['Final_forecast_N_Mesi_'+lag+str(n_mesi)])
                                          /dataframe['Final_forecast_N_Mesi_'+lag+str(n_mesi)]).round(3)
    
    return dataframe

**Individuazione del miglior numero di mesi di train per ogni combinazione(dataframe) e scelta(automatica) di questi**

In [ ]:
def makeListBest(dataframe_prediction, n_mesi_max):
    anni = ['2020','2021','2022']
    delta_assoluto = 7
    salta = 4
    n_mesi_best_somma = pd.DataFrame(columns=['Mesi_Best_Somma_Ass','2020','2021','2022','Somma_Delta_Assoluto'])
    n_mesi_best_mean = pd.DataFrame(columns=['Mesi_Best_Media_Ass','2020','2021','2022','Media_Delta_Assoluto'])
    n_mesi_best_mean_perc = pd.DataFrame(columns=['Mesi_Best_Media_Perc','Media_Delta_Percentuale'])
    colonne = list(range(delta_assoluto+1))
    
    i=0
    somma = list()
    mean = list()
    mean_perc = list()
    for dataframe in dataframe_prediction:
        print("Dataframe "+str(i))
        
        final = dataframe.filter(regex='Delta_ass_finale')
        final_perc = dataframe.filter(regex='Delta_perc_nuovo')
        final_perc.replace([np.inf, -np.inf], 0, inplace=True)

        dataframe = dataframe.iloc[:,colonne]
        dataframe = pd.concat([dataframe,final], axis = 1)
        dataframe.rename(columns = {'Delta_ass_old' : 'Delta_ass_old_00'}, inplace = True)
        somma.append(pd.DataFrame(columns=['Colonna','2020','2021','2022','Somma_Delta_Assoluto']))
        mean.append(pd.DataFrame(columns=['Colonna','2020','2021','2022','Media_Delta_Assoluto']))
        mean_perc.append(pd.DataFrame(columns=['Colonna','Media_Delta_Percentuale']))
        #display(dataframe)
        y = 0
        for colonna in dataframe.columns[delta_assoluto+1:]:
            tot_delta = list()
            mean_delta = list()
            tot_tot_delta = 0
            mean_mean_delta = 0
            for anno in range(len(anni)):
                tot_delta.append(dataframe[colonna][anni[anno]].sum())
                mean_delta.append(dataframe[colonna][anni[anno]].mean())
                tot_tot_delta += tot_delta[anno]
                mean_mean_delta += mean_delta[anno]
            #display(dataframe[colonna][y:])
            mean_mean_delta = dataframe[colonna][y:].mean()
            y += 1
            newline_somma = {'Colonna' : colonna, '2020': tot_delta[0], '2021': tot_delta[1], '2022': tot_delta[2], 'Somma_Delta_Assoluto': tot_tot_delta}
            newline_mean = {'Colonna' : colonna, '2020': mean_delta[0], '2021': mean_delta[1], '2022': mean_delta[2], 'Media_Delta_Assoluto': mean_mean_delta}
            somma[i] = somma[i].append(newline_somma, ignore_index=True)
            mean[i] = mean[i].append(newline_mean, ignore_index=True)
        x = 0
        for colonna in final_perc.columns:
            #print(final_perc[colonna][x:])
            mean_percentuale = final_perc[colonna][x:].mean()
            x += 1
            newline_mean_perc = {'Colonna' : colonna, 'Media_Delta_Percentuale' : mean_percentuale}
            mean_perc[i] = mean_perc[i].append(newline_mean_perc, ignore_index=True)
        
        somma[i].set_index('Colonna', inplace = True)
        display(somma[i])
        min_somma = somma[i][somma[i].Somma_Delta_Assoluto.abs() == somma[i].Somma_Delta_Assoluto.abs().min()]
        min_somma = min_somma.reset_index()
        display(min_somma)
        mese = min_somma.iloc[0,0][-1]
        
        mean[i].set_index('Colonna', inplace = True)
        display(mean[i])
        min_mean = mean[i][mean[i].Media_Delta_Assoluto.abs() == mean[i].Media_Delta_Assoluto.abs().min()]
        min_mean = min_mean.reset_index()
        display(min_mean)

        mean_perc[i].set_index('Colonna', inplace = True)
        display(mean_perc[i])
        min_mean_perc = mean_perc[i][mean_perc[i].Media_Delta_Percentuale.abs() == mean_perc[i].Media_Delta_Percentuale.abs().min()]
        min_mean_perc = min_mean_perc.reset_index()
        display(min_mean_perc)
        
        mese_somma = min_somma.iloc[0,0][-1]
        if (mese_somma == '0' or mese_somma == '1' or mese_somma == '2'):
            mese_somma = min_somma.iloc[0,0][-2:]
        newline_somma = {'Mesi_Best_Somma_Ass' : mese_somma, '2020': min_somma.iloc[0,1], '2021': min_somma.iloc[0,2], '2022': min_somma.iloc[0,3], 'Somma_Delta_Assoluto': min_somma.iloc[0,4]}
        n_mesi_best_somma = n_mesi_best_somma.append(newline_somma, ignore_index=True)
        
        mese_mean = min_mean.iloc[0,0][-1]
        if (mese_mean == '0' or mese_mean == '1' or mese_mean == '2'):
            mese_mean = min_mean.iloc[0,0][-2:]
        newline_mean = {'Mesi_Best_Media_Ass' : mese_mean, '2020': min_mean.iloc[0,1], '2021': min_mean.iloc[0,2], '2022': min_mean.iloc[0,3], 'Media_Delta_Assoluto': min_mean.iloc[0,4]}
        n_mesi_best_mean = n_mesi_best_mean.append(newline_mean, ignore_index=True)

        mese_mean_perc = min_mean_perc.iloc[0,0][-1]
        if (mese_mean_perc == '0' or mese_mean_perc == '1' or mese_mean_perc == '2'):
            mese_mean_perc = min_mean_perc.iloc[0,0][-2:]
        newline_mean_perc = {'Mesi_Best_Media_Perc' : mese_mean_perc, 'Media_Delta_Percentuale': min_mean_perc.iloc[0,1]}
        n_mesi_best_mean_perc = n_mesi_best_mean_perc.append(newline_mean_perc, ignore_index=True)
        
        i += 1
    display(n_mesi_best_somma)
    mesi_best_somma = n_mesi_best_somma.Mesi_Best_Somma_Ass.to_list()
    display(n_mesi_best_mean)
    mesi_best_mean = n_mesi_best_mean.Mesi_Best_Media_Ass.to_list()
    display(n_mesi_best_mean_perc)
    mesi_best_mean_perc = n_mesi_best_mean_perc.Mesi_Best_Media_Perc.to_list()
    return n_mesi_best_somma, n_mesi_best_mean, n_mesi_best_mean_perc, mesi_best_somma, mesi_best_mean, mesi_best_mean_perc

**Creazione dei plot di confronto delle vendite e predizioni per ogni combinazione**

In [ ]:
def makePlot(df_finale, n_mesi = 3, lag = '', n_dataframe = 0, metrica = 'SUM'):    
    '''
    Una funzione per creare un plot specifico ad ogni dataframe che viene dato come input  
    '''
    
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(111)

    ax1.plot(df_finale['Vendite_programmate']/1000,
    linestyle='-',
    linewidth=2,
    label='Vendite programmate')
    
    ax1.legend(frameon=False)

    ax1.plot(df_finale['Vendite_consuntive']/1000,
         linestyle='-',
         linewidth=2,
         label='Vendite consuntive')

    ax1.legend(frameon=False)

    ax1.plot(df_finale['Final_forecast_N_Mesi_'+lag+str(n_mesi)]/1000,
         linestyle='-',
         color='black',
         linewidth=2,
         label='Vendite previste')
    
    ax1.legend(frameon=False)
    
    ax1.set_ylabel('Vendite (kT)')   
    ax1.set_xlabel('Tempo')

    x = 0
    divisione = 0
    while divisione == 0:
        divisione = df_finale['Divisione'].values[x] 
        cliente = df_finale['Cliente'].values[x]
        sub_pool= df_finale['Sub_pool'].values[x]
        mezzo_trasporto = df_finale['Mezzo_trasporto'].values[x]
        funzione_commerciale = df_finale['Funzione_commerciale'].values[x]
        x += 1

    divisione= divisione.capitalize()
    cliente= cliente.capitalize()
    sub_pool= sub_pool.capitalize()
    mezzo_trasporto = mezzo_trasporto.capitalize()
    funzione_commerciale = funzione_commerciale.capitalize()
       
    plt.title(f'2020-2022 - Combinazione:  " {divisione} - {cliente} - {sub_pool} - {mezzo_trasporto} - {funzione_commerciale}" \n Numero mesi train '+lag+': '+str(n_mesi) + ' , Metrica : '+metrica, fontsize=12)
    plt.inferno()
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Università/Master/Grafici/GraficiPrevisioniCombinazione'+metrica+lag+str(n_dataframe))
    plt.show()

**Ottenimento del risultato finale**

In [ ]:
i = 0
d_pred = list()
N_MESI_LAGGED_MAX = 3
N_MESI_TRAIN_MAX = 8

#Scelti i mesi di lag e di train, si applica la funzione makeForecast che contiene tutte le altre al 
#fine di ottenere una predizione
#i primi 6 mesi vengono rimosssi perchè in ogni forecast almeno abbiamo 3 mesi di lag + 3 di train
#questi ultimi(mesi di train) aumentano in base alla scelta del numero massimo scelto

for dataframe in df_comb_groupby:
    dataframe = dataframe[['Divisione','Sub_pool','Cliente','Mezzo_trasporto', 'Funzione_commerciale',
                           'Vendite_programmate','Vendite_consuntive']]
    dataframe['Delta_ass_old']=(dataframe['Vendite_consuntive']-dataframe['Vendite_programmate'])
    dataframe['Delta_perc_old']=((dataframe['Vendite_consuntive']-dataframe['Vendite_programmate'])
                                            /dataframe['Vendite_programmate']).round(3)
    
    dataframe = makeForecast(dataframe, n_mesi_max = N_MESI_TRAIN_MAX)       
    dataframe = dataframe.iloc[6:,:]
    display(dataframe)
    d_pred.append(dataframe)
    i += 1

In [ ]:
N_MESI_TRAIN_MAX = 8
score_sum, score_mean, score_mean_perc, mesi_best_sum, mesi_best_mean, mesi_best_perc = makeListBest(d_pred, N_MESI_TRAIN_MAX)

**Plot dei risultati (3 lag)**

In [ ]:
print(mesi_best_sum)
mesi_best = mesi_best_sum.copy()
x = 0
for dataframe in d_pred:
    mese = mesi_best.pop(0)
    makePlot(dataframe,mese, n_dataframe=x)
    x += 1

In [ ]:
print(mesi_best_mean)
mesi_best = mesi_best_mean.copy()
x = 0
for dataframe in d_pred:
    mese = mesi_best.pop(0)
    makePlot(dataframe,mese, n_dataframe = x, metrica = 'MEAN')
    x += 1

In [ ]:
score_ass = pd.concat([score_sum, score_mean], axis = 1)
display(score_ass)
score = pd.concat([score_ass, score_mean_perc], axis = 1)
score_redu = score[['Mesi_Best_Somma_Ass', 'Mesi_Best_Media_Ass', 'Mesi_Best_Media_Perc', 'Somma_Delta_Assoluto', 'Media_Delta_Assoluto', 'Media_Delta_Percentuale']]
score_redu
#dfi.export(score_redu, '/content/drive/MyDrive/Università/Master/Grafici/Score_Best.png')

**In questo caso vengono variati i lag e vengono tenuti costanti i mesi di train**

In [ ]:
i = 0
d_pred_lag = list()
N_MESI_LAGGED_MAX = 12
N_MESI_TRAIN_MAX = 3

#tale parte di codice è stata aggiunta per dare la massima libertà di personalizzazione della soluzione 
#così da poter non solo scegliere di variare i mesi di train, ma pure i lag, anche se dall'acf è stato visto che 3 sembra
#essere il numero perfetto di lag

for dataframe in df_comb_groupby:
    dataframe = dataframe[['Divisione','Sub_pool','Cliente','Mezzo_trasporto', 'Funzione_commerciale',
                           'Vendite_programmate','Vendite_consuntive']]
    dataframe['Delta_ass_old']=(dataframe['Vendite_consuntive']-dataframe['Vendite_programmate'])
    dataframe['Delta_perc_old']=((dataframe['Vendite_consuntive']-dataframe['Vendite_programmate'])
                                            /dataframe['Vendite_programmate']).round(3)
    
    dataframe = makeForecast(dataframe, n_mesi_max = N_MESI_LAGGED_MAX, lag='Lag_')       
    #print('dataframe_pred')
    #display(dataframe_pred)
    dataframe = dataframe.iloc[6:,:]
    d_pred_lag.append(dataframe)
    i += 1

In [ ]:
N_MESI_LAGGED_MAX = 12
score_sum_lag, score_mean_lag, score_mean_perc_lag, mesi_best_sum_lag, mesi_best_mean_lag, mesi_best_perc_lag = makeListBest(d_pred_lag, N_MESI_LAGGED_MAX)

In [ ]:
print(mesi_best_sum_lag)
mesi_best = mesi_best_sum_lag.copy()
x = 0
for dataframe in d_pred_lag:
    mese = mesi_best.pop(0)
    makePlot(dataframe, mese, lag = 'Lag_', n_dataframe=x)
    x += 1

In [ ]:
print(mesi_best_mean_lag)
mesi_best = mesi_best_mean_lag.copy()
x = 0
for dataframe in d_pred_lag:
    mese = mesi_best.pop(0)
    makePlot(dataframe,mese, n_dataframe = x, lag = 'Lag_', metrica = 'MEAN')
    x += 1

**Visione dei differenti punteggi al variare dei mesi di lag e di train**

In [ ]:
score_ass_lag = pd.concat([score_sum_lag, score_mean_lag], axis = 1)
display(score_ass_lag)
score_lag = pd.concat([score_ass_lag, score_mean_perc_lag], axis = 1)
score_redu_lag = score[['Mesi_Best_Somma_Ass', 'Mesi_Best_Media_Ass', 'Mesi_Best_Media_Perc', 'Somma_Delta_Assoluto', 'Media_Delta_Assoluto', 'Media_Delta_Percentuale']]
score_redu_lag

In [ ]:
def makePlotDelta(df_finale, n_mesi = 3, lag = '', n_dataframe = 0, metrica = 'SUM'):    
    '''
    Una funzione per creare un plot specifico ad ogni dataframe che viene dato come input  
    '''
    
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(111)

    ax1.plot(df_finale['Delta_ass_old']/1000,
    linestyle='-',
    linewidth=2,
    label='Delta Assoluto Vecchio')
    
    ax1.legend(frameon=False)


    ax1.plot(df_finale['Delta_ass_finale_N_Mesi_'+lag+str(n_mesi)]/1000,
         linestyle='-',
         #color='black',
         linewidth=2,
         label='Delta Assoluto Soluzione')
    
    ax1.legend(frameon=False)
    
    ax1.set_ylabel('Delta Assoluto Vendite [kT]')   
    ax1.set_xlabel('Tempo')

    x = 0
    divisione = 0
    while divisione == 0:
        divisione = df_finale['Divisione'].values[x] 
        cliente = df_finale['Cliente'].values[x]
        sub_pool= df_finale['Sub_pool'].values[x]
        mezzo_trasporto = df_finale['Mezzo_trasporto'].values[x]
        funzione_commerciale = df_finale['Funzione_commerciale'].values[x]
        x += 1

    divisione= divisione.capitalize()
    cliente= cliente.capitalize()
    sub_pool= sub_pool.capitalize()
    mezzo_trasporto = mezzo_trasporto.capitalize()
    funzione_commerciale = funzione_commerciale.capitalize()
       
    plt.title(f'2020-2022 - Combinazione:  " {divisione} - {cliente} - {sub_pool} - {mezzo_trasporto} - {funzione_commerciale}" ', fontsize=12)
    plt.inferno()
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Università/Master/Grafici/GraficiDeltaAssoluto'+metrica+lag+str(n_dataframe))
    plt.show()

In [ ]:
print(mesi_best_mean)
mesi_best = mesi_best_mean.copy()
x = 0
for dataframe in d_pred:
    mese = mesi_best.pop(0)
    makePlotDelta(dataframe, 3, n_dataframe = x, metrica = 'MEAN')
    x += 1